In [52]:
import pandas as pd
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
import matplotlib.pyplot as plt
import seaborn as sns
import os
from preprocessing import Feature_Extractor, data_preprocessing


In [53]:
symbol = "BID"
predictest = pd.read_csv(f'{symbol}_LSTMAttention_evaluate_result.csv')

In [54]:
predictest.dtypes

Unnamed: 0     int64
Date          object
predicted     object
dtype: object

In [55]:
def convert_to_float(x):
    return float(x[0])

In [56]:
if predictest['predicted'].apply(lambda x: isinstance(x, str)).any():
    predictest['predicted'] = predictest['predicted'].apply(eval).apply(convert_to_float)
predictest.dtypes

Unnamed: 0      int64
Date           object
predicted     float64
dtype: object

In [57]:
#Lưu file mới
predictest = predictest.drop(columns=['Unnamed: 0'], errors='ignore')
output_file_path = os.path.join(f'{symbol}_LSTMAttention_evaluate_result.csv')
predictest.to_csv(output_file_path, index=False)

In [58]:
df = pd.read_csv(f'DataTrainLSTM/{symbol}.csv')
df.head


<bound method NDFrame.head of             Date    Close     Open     High      Low
0     2019-01-28  24414.4  24529.3  24758.9  24337.9
1     2019-01-29  24261.4  24414.4  24414.4  24146.6
2     2019-01-30  24261.4  24414.4  24682.3  24184.8
3     2019-01-31  24108.3  24644.1  24644.1  24108.3
4     2019-02-01  24184.8  24070.0  24376.2  24031.8
...          ...      ...      ...      ...      ...
1327  2024-05-27  48900.0  49400.0  49400.0  48550.0
1328  2024-05-28  49100.0  49000.0  49200.0  48600.0
1329  2024-05-29  48600.0  49450.0  49450.0  48600.0
1330  2024-05-30  47700.0  48400.0  48500.0  47400.0
1331  2024-05-31  47100.0  47700.0  48300.0  47100.0

[1332 rows x 5 columns]>

In [59]:
# Hợp nhất hai tệp CSV theo cột Date
df_merged = pd.merge(df, predictest, on='Date', how='left')
df_merged['Close'] = df_merged['predicted'].combine_first(df_merged['Close'])
df_merged.drop(columns=['predicted'], inplace=True)
cutoff_date = '2024-05-17'
df_filtered = df_merged[df_merged['Date'] <= cutoff_date]
output_folder = 'PredictTestData'
output_file_path = os.path.join(output_folder, f'{symbol}LSTMAttentionpredictions.csv')
df_filtered.to_csv(output_file_path, index=False)